In [2]:
import importlib
import data
from gurobipy import *

# get data from excel
dataframes = data.get_all_pandas()

LOADING DATA: 7it [01:13,  7.30s/it]                       

In [6]:
import variables
import contraintes
import importlib
importlib.reload(variables) #to remive in final .py
importlib.reload(contraintes) #to remive in final .py

<module 'contraintes' from 'c:\\Users\\remgo\\Desktop\\ProjetST7\\git\\contraintes.py'>

In [4]:
from openpyxl import Workbook
def show_solution(model):
    # print(var_dict)
    print("Objectif minimal: ", m.objVal)
    #build excel of tasks
    workbook = Workbook()
    sheet = workbook.active

    sheet["A1"] = "Id_tache"
    sheet["B1"] = "tache"
    sheet["C1"] = "minute"
    row = 2
    for tache in var_dict.keys():
        for sillon_id in var_dict[tache].keys():
            sheet[f"B{row}"] = tache
            sheet[f"C{row}"] = var_dict[tache][sillon_id].X
            sheet[f"A{row}"] = sillon_id

            row += 1


    workbook.save(filename="results/hello_world.xlsx")
    


In [7]:
# create model
m = Model()

# generate variables (integers)
var_dict = variables.generate_variables(m, dataframes)
# generate variables
contraintes.generate_contraintes(m, dataframes, var_dict)

final_train_task = dataframes["sillons_df"]
final_train_task = final_train_task[final_train_task["JARR"]==final_train_task["JARR"].max()]
final_train_task = final_train_task[final_train_task["HDEP"]==final_train_task["HDEP"].max()]["train_id"].iloc[0]
final_train_task = var_dict["essai de frein départ"][final_train_task]
print(final_train_task)
m.setObjective(final_train_task, GRB.MINIMIZE)

m.update()
m.printStats()
m.presolve()
m.printStats()
m.optimize()
m.write("test.lp")
show_solution(m)

Initialising variables: 100%|██████████| 217/217 [00:00<00:00, 302.34it/s]
c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:106: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tache_precedante = taches_df[taches_df["Ordre"] == ordre -
c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:145: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chantier_cycles[chantier, "start"] = taches_df[taches_df["Chantier"] == chantier][taches_df.Ordre ==
c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:147: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chantier_cycles[chantier, "end"] = taches_df[taches_df["Chantier"] == chantier][taches_df.Ordre ==


Adding occupation constrainst for: WPY_REC...
Adding occupation constrainst for: WPY_DEP...
Adding occupation constrainst for: WPY_FOR...
<gurobi.Var *Awaiting Model Update*>

Statistics for modelUnnamed:
  Linear constraint matrix    : 69803 Constrs, 105136 Vars, 277486 NZs
  Variable types              : 0 Continuous,
105136 Integer (104056 Binary)
  General constraints         : 173642 Constrs
  Matrix coefficient range    : [ 1, 1e+06 ]
  Objective coefficient range : [ 1, 1 ]
  Variable bound range        : [ 1, 1 ]
  RHS coefficient range       : [ 14, 999985 ]
Presolve removed 2478 rows and 3124 columns (presolve time = 5s) ...
Presolve removed 2478 rows and 3124 columns (presolve time = 21s) ...
Presolve added 240750 rows and 240104 columns
Presolve time: 21.48s

Statistics for modelUnnamed:
  Linear constraint matrix    : 69803 Constrs, 105136 Vars, 277486 NZs
  Variable types              : 0 Continuous,
105136 Integer (104056 Binary)
  General constraints         : 173642 Co

In [ ]:
# tests 
importlib.reload(contraintes)
importlib.reload(variables)
# create model
m = Model()

# generate variables (integers)
var_dict = variables.generate_variables(m, dataframes)
# generate variables
contraintes.generate_contraintes(m, dataframes, var_dict)

Initialising variables: 100%|██████████| 217/217 [00:00<00:00, 889.35it/s]
c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:104: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  1][taches_df["Type de train"] == type_train]["Type de tache humaine"].iloc[0]


540
Adding occupation constrainst for: WPY_DEP...


c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:150: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  taches_df[taches_df["Chantier"] == chantier].Ordre.max()]["Type de tache humaine"].iloc[0]
c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  taches_df[taches_df["Chantier"] == chantier].Ordre.min()]["Type de tache humaine"].iloc[0]


Adding occupation constrainst for: WPY_REC...
Adding occupation constrainst for: WPY_FOR...
